In [1]:
import pandas as pd
import numpy as np
import re
from difflib import SequenceMatcher
import time

In [2]:
dog = pd.read_csv('data/dogs.csv')
dog_travel = pd.read_csv('data/dogTravel.csv')
nst = pd.read_csv('data/NST-EST2021-POP.csv', header=None, names=['state','population'])

### Reshaping the dataset

In [3]:
pd.set_option('max_columns', None)

In [4]:
# there are some rows with shifted value, means that some rows of column 
# contact_state contains the values of the contact_zip
# so to identiy all misplaced values I decided to pick a column where all values
# were supposed to be letters and with regex i identified all rows that contains digits
w_subset = dog[dog['contact_state'].str.contains(r'\d') == True]
# i also create another variables with all the right rows which I'll use leter for merging 
r_subset = dog[dog['contact_state'].str.contains(r'\d') == False]

#subset.reset_index(drop= True, inplace=True)

In [5]:
w_subset.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,NaN,Young,Male,Large,NaN,False,False,NaN,False,False,NaN,NaN,NaN,"Gunther :Gunny\"",Meet handsome 3 year old Gunt...",2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20,NaN,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,White / Cream,NaN,Adult,Female,Large,Short,True,True,NaN,False,True,NaN,NaN,False,"ANNABELLE \ANNIE\"",""You can fill out an adopti...",2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20,NaN,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,True,True,True,"PEPPER \Courtesy listing\"",""This is Pepper. He...",2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20,NaN,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,NaN,NaN,False,"COOPER \courtesy listing\"",""Cooper is 13 years...",2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20,NaN,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,White / Cream,NaN,Senior,Female,Medium,Short,True,True,NaN,False,True,False,False,NaN,"DAISY \courtesy listing\"",""•Basset Hound, fema...",2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20,NaN,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61


In [7]:
# good and good 2 contains the right columns from the subset with the shifted rows
good = w_subset.iloc[:, 0:25]
good2 = w_subset.iloc[:, 33:37]

# bad contains the misplaced columns
bad = w_subset.iloc[:, 25:33]
# i shift the values of a column 
bad = bad.shift(periods=1, axis=1)
# i concatenate all datasets in a new one
new = pd.concat([good, bad, good2], axis=1)


In [8]:
new.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,NaN,Young,Male,Large,NaN,False,False,NaN,False,False,NaN,NaN,NaN,"Gunther :Gunny\"",Meet handsome 3 year old Gunt...",NaN,2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,White / Cream,NaN,Adult,Female,Large,Short,True,True,NaN,False,True,NaN,NaN,False,"ANNABELLE \ANNIE\"",""You can fill out an adopti...",NaN,2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,True,True,True,"PEPPER \Courtesy listing\"",""This is Pepper. He...",NaN,2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,NaN,NaN,False,"COOPER \courtesy listing\"",""Cooper is 13 years...",NaN,2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,White / Cream,NaN,Senior,Female,Medium,Short,True,True,NaN,False,True,False,False,NaN,"DAISY \courtesy listing\"",""•Basset Hound, fema...",NaN,2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61


In [9]:
# now i put the new dataset with the shifted values into the original one
dog = pd.concat([r_subset, new], ignore_index=True, axis=0)

In [10]:
dog.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,Yellow / Tan / Blond / Fawn,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,NaN,NaN,NaN,HARLEY,adoptable,2019-09-20T16:37:59+0000,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,White / Cream,NaN,Adult,Male,Large,Short,True,True,NaN,False,True,NaN,NaN,NaN,BIGGIE,adoptable,2019-09-20T16:24:57+0000,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,NaN,NaN,Adult,Male,Large,Short,True,False,NaN,False,True,NaN,NaN,NaN,Ziggy,adoptable,2019-09-20T14:10:11+0000,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,NaN,Baby,Female,Large,NaN,False,False,NaN,False,False,NaN,NaN,NaN,Gypsy,adoptable,2019-09-20T10:08:22+0000,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,NaN,NaN,Young,Male,Small,Long,True,False,NaN,False,True,True,True,True,Theo,adoptable,2019-09-20T06:48:30+0000,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


## 1. Extract all dogs with status that is not adoptable

In [11]:
dog['status'].unique()

array(['adoptable', nan], dtype=object)

In [12]:
dog['status'].value_counts(dropna=False)

adoptable    58147
NaN             33
Name: status, dtype: int64

In [13]:
not_adopt_dog = dog[dog['status'].isnull()]
not_adopt_dog.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
58147,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,NaN,Young,Male,Large,NaN,False,False,NaN,False,False,NaN,NaN,NaN,"Gunther :Gunny\"",Meet handsome 3 year old Gunt...",NaN,2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
58148,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,White / Cream,NaN,Adult,Female,Large,Short,True,True,NaN,False,True,NaN,NaN,False,"ANNABELLE \ANNIE\"",""You can fill out an adopti...",NaN,2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20,Dog,You can fill out an adoption application onlin...,80,130.77
58149,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,True,True,True,"PEPPER \Courtesy listing\"",""This is Pepper. He...",NaN,2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
58150,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,NaN,NaN,False,"COOPER \courtesy listing\"",""Cooper is 13 years...",NaN,2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
58151,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,White / Cream,NaN,Senior,Female,Medium,Short,True,True,NaN,False,True,False,False,NaN,"DAISY \courtesy listing\"",""•Basset Hound, fema...",NaN,2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61


In [14]:
not_adopt_dog.shape

(33, 37)

## 2. For each (primary) breed, determine the number of dogs

In [15]:
group_breed = dog.groupby('breed_primary').count()['id'].reset_index()
group_breed.columns = ['breed_primary', 'number_dogs']
group_breed.head()

,breed_primary,number_dogs
0,Affenpinscher,17
1,Afghan Hound,4
2,Airedale Terrier,19
3,Akbash,3
4,Akita,181


## 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

In [16]:
dog.isnull().sum()

id                     0
org_id                 0
url                    0
type.x                 0
species                0
breed_primary          0
breed_secondary    37359
breed_mixed            0
breed_unknown          0
color_primary      32046
color_secondary    46121
color_tertiary     56963
age                    0
sex                    0
size                   0
coat               30995
fixed                  0
house_trained          0
declawed           58180
special_needs          0
shots_current          0
env_children       30153
env_dogs           23511
env_cats           38828
name                   0
status                33
posted                 0
contact_city           0
contact_state          0
contact_zip           12
contact_country        0
stateQ                 0
accessed               0
type.y               640
description         8705
stay_duration          0
stay_cost              0
dtype: int64

In [17]:
dog[dog['breed_secondary'].isnull()]['breed_mixed'].value_counts(normalize=True)

True     0.555957
False    0.444043
Name: breed_mixed, dtype: float64

In [18]:
# i create a copy of the main dataframe
new_df = dog.copy()
# i decided to drop the null values from the breed_secondary column
new_df.dropna(subset = ['breed_secondary'], inplace=True)

# based on the value of the breed secondary this function return if it's mixed or not mixed breed 
def function(row):
    if row['breed_secondary'] == 'Mixed Breed':
        return 'Mixed'
    if row['breed_secondary'] != 'Mixed Breed':
        return 'Not Mixed'

# i apply the function to the dataset
new_df['mixed_or_not'] = new_df.apply(lambda row: function(row), axis=1)

In [19]:
perc_df = pd.DataFrame(round(new_df.groupby('breed_primary')['mixed_or_not'].value_counts() / new_df.groupby('breed_primary')['id'].count(),2) * 100, columns=['percentage'])
perc_df.head()

percentage
breed_primary    mixed_or_not            
Affenpinscher    Mixed               50.0
                 Not Mixed           50.0
Afghan Hound     Not Mixed          100.0
Airedale Terrier Not Mixed           89.0
                 Mixed               11.0

## 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

In [20]:
# formatting the types in the dataset into datatime type
dog.posted = pd.to_datetime(dog.posted)
dog.accessed = pd.to_datetime(dog.accessed)

In [21]:
dog.groupby('breed_primary')['posted'].min()

breed_primary
Affenpinscher                       2012-03-08 10:27:33+00:00
Afghan Hound                        2017-06-29 23:28:51+00:00
Airedale Terrier                    2014-06-13 12:59:36+00:00
Akbash                              2019-07-21 00:35:59+00:00
Akita                               2012-03-03 09:31:08+00:00
                                               ...           
Wirehaired Pointing Griffon         2016-06-29 20:03:55+00:00
Wirehaired Terrier                  2012-11-27 14:07:54+00:00
Xoloitzcuintli / Mexican Hairless   2007-02-01 00:00:00+00:00
Yellow Labrador Retriever           2010-05-31 00:00:00+00:00
Yorkshire Terrier                   2012-02-04 17:14:52+00:00
Name: posted, Length: 216, dtype: datetime64[ns, UTC]

In [22]:
dog.groupby('breed_primary')['posted'].max()

breed_primary
Affenpinscher                       2019-09-14 10:10:51+00:00
Afghan Hound                        2019-07-27 00:38:48+00:00
Airedale Terrier                    2019-09-19 18:40:39+00:00
Akbash                              2019-08-23 17:11:04+00:00
Akita                               2019-09-20 15:19:57+00:00
                                               ...           
Wirehaired Pointing Griffon         2016-06-29 20:03:55+00:00
Wirehaired Terrier                  2019-09-19 22:52:45+00:00
Xoloitzcuintli / Mexican Hairless   2019-09-08 11:15:54+00:00
Yellow Labrador Retriever           2019-09-20 06:30:27+00:00
Yorkshire Terrier                   2019-09-20 16:56:38+00:00
Name: posted, Length: 216, dtype: datetime64[ns, UTC]

## 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [23]:
# function that return the difference between male and female dogs
def difference(dog):
    difference = dog['sex'].value_counts()['Male'] - dog['sex'].value_counts()['Female']
    return difference

In [24]:
imbalance = dog.groupby('contact_state').apply(difference).reset_index()
imbalance.columns = ['contact_state', 'sex_difference']
# column with the absolute value on which i compute the max
imbalance['sex_difference_abs'] = imbalance.sex_difference.abs()

In [25]:
imbalance[imbalance['sex_difference_abs']== imbalance['sex_difference_abs'].max()]

,contact_state,sex_difference,sex_difference_abs
36,OH,205,205


Ohio is the state with the highest sex imbalance 

## 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [26]:
dog.groupby(['age','size'])[['stay_duration','stay_cost']].mean()

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

## 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [27]:
# to find the dog with 3 or more travels I use the id
count_id = pd.DataFrame(dog_travel['id'].value_counts())
count_id.reset_index(inplace=True)
subset_count_id = count_id[count_id['id']>=3]
subset_count_id.head()
subset_count_id.columns = ['id', 'travel_count']
# subset_count_id contains all id that repeat themselves more that 3 times 

In [28]:
# to list the breeds of those dogs i need to merge subset_count_id with the main dog dataset that has the breed information
breed_travel = pd.merge(dog[['id', 'breed_primary']], subset_count_id, on = 'id', how='inner')
breed_travel.head()

,id,breed_primary,travel_count
0,45642530,Jindo,4
1,46039420,Border Collie,4
2,40036107,Pit Bull Terrier,3
3,45851842,Labrador Retriever,3
4,45841145,Mixed Breed,3


In [29]:
breed_travel.shape

(563, 3)

## 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

In [30]:
# I used an np.where condition 
dog_travel['found'] = np.where(dog_travel['manual'].isnull()==False,dog_travel['manual'],dog_travel['found'])

In [31]:
dog_travel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


## 9. For each state, compute the ratio between the number of travels and the population.

In [32]:
# some rows contain a code instead of the contact state initials 
# so i use loc to replace them
dog_travel.loc[dog_travel['contact_state'] == '17325', 'contact_state'] = 'PA'

In [33]:
# aggiungo un'altra colonna con la sigla per ogni stato al dataset che contiene i dati della popolatione
nst['contact_state'] = ['AL', 'AK', 'AZ','AR', 'CA', 'CO', 'CT', 'DE', 'DC',
 'FL', 'GA', 'HI', 'ID', 'IL','IN', 
 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
 'MN','MS', 'MO', 'MT', 'NE','NV', 'NH', 'NJ','NM', 
 'NY','NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
 'TN', 'TX', 'UT', 'VT', 'VA','WA', 'WV', 'WI','WY']

In [34]:
# raggruppo per stato e faccio un count e rinonimo le colonne
travel_state_count = dog_travel.groupby('contact_state')['id'].count().reset_index()
travel_state_count.columns = ['contact_state', 'count']
# unisco i dataset con merge con argomento inner per tenere solo gli stati che hanno dei travel
travel_state_count = pd.merge(nst,travel_state_count, on = 'contact_state', how='inner')
# sistemo la colonna popolazione toglieno i punti tra i numeri e trasformo il tutto in intger
travel_state_count = travel_state_count.replace(to_replace=r'\.',value= '',regex=True)
travel_state_count['population'] = travel_state_count['population'].astype('int')
# aggiungo una colonna con il ratio
travel_state_count['ratio'] = travel_state_count['count'] / travel_state_count['population'] *100
travel_state_count

,state,population,contact_state,count,ratio
0,Alabama,5024279,AL,75,0.001493
1,Arizona,7151502,AZ,70,0.000979
2,Arkansas,3011524,AR,10,0.000332
3,California,39538223,CA,28,0.000071
4,Colorado,5773714,CO,103,0.001784
5,Connecticut,3605944,CT,90,0.002496
6,Delaware,989948,DE,57,0.005758
7,District of Columbia,689545,DC,112,0.016243
8,Florida,21538187,FL,133,0.000618
9,Georgia,10711908,GA,109,0.001018


## 10. For each dog, compute the number of days from the posted day to the day of last access

In [35]:
# formatto le colonne con giorno mese e anno 
# inoltre salvo un'altra colonna come posted_day in cui tolgo l'orario
dog['posted_day'] = dog['posted'].dt.strftime('%Y/%m/%d')
dog['accessed'] = dog['accessed'].dt.strftime('%Y/%m/%d')

In [36]:
dog[['accessed','posted_day']] = dog[['accessed','posted_day']].apply(pd.to_datetime)
dog['days_difference'] = (dog['accessed'] - dog['posted_day']).dt.days

In [37]:
dog.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,posted_day,days_difference
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,Yellow / Tan / Blond / Fawn,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,NaN,NaN,NaN,HARLEY,adoptable,2019-09-20 16:37:59+00:00,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81,2019-09-20,0
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,White / Cream,NaN,Adult,Male,Large,Short,True,True,NaN,False,True,NaN,NaN,NaN,BIGGIE,adoptable,2019-09-20 16:24:57+00:00,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07,2019-09-20,0
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,NaN,NaN,Adult,Male,Large,Short,True,False,NaN,False,True,NaN,NaN,NaN,Ziggy,adoptable,2019-09-20 14:10:11+00:00,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51,2019-09-20,0
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,NaN,Baby,Female,Large,NaN,False,False,NaN,False,False,NaN,NaN,NaN,Gypsy,adoptable,2019-09-20 10:08:22+00:00,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83,2019-09-20,0
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,NaN,NaN,Young,Male,Small,Long,True,False,NaN,False,True,True,True,True,Theo,adoptable,2019-09-20 06:48:30+00:00,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09,2019-09-20,0


## 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

In [38]:
dog['week_difference'] = (dog['days_difference'] / 7).apply(np.floor)

In [39]:
dog['week_difference'].value_counts().reset_index().rename(columns = {'index': 'week_difference', 'week_difference':'count'})

,week_difference,count
0,0.0,9803
1,1.0,6547
2,2.0,5764
3,3.0,3353
4,4.0,2439
...,...,...
574,484.0,1
575,646.0,1
576,439.0,1
577,598.0,1


## 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [40]:
# subset del dataset con le stesse razze primarie e sesso 
dog_dupl = dog[dog.duplicated(subset=['breed_primary','sex'])]
# reset dell'indice
dog_dupl.reset_index(drop=True, inplace=True)

In [41]:
# droppo le righe che hanno nulla la descrizione e resetto l'indice
dog_dupl = dog_dupl.dropna(subset=['description'])
dog_dupl.reset_index(drop=True, inplace=True)

In [42]:
# rendo tutte le parole nel campo descrizione minuscole
dog_dupl['description'] = dog_dupl['description'].str.lower()
# sostituisco tutti i caratteri speciali con uno spazio vuoto 
dog_dupl['description'] = dog_dupl['description'].str.replace('[^A-Za-z0-9]+', ' ')
dog_dupl['description'] = dog_dupl['description'].str.split(r"\s+")
dog_dupl['description'] = dog_dupl['description'].str.join(' ')

In [43]:
index, col = dog_dupl.shape
start_time = time.time()
f_list = []
for ind in range(index-1):
    if SequenceMatcher(None, dog_dupl['description'][ind] ,dog_dupl['description'][ind+1]).ratio() >= 0.9:
        f_list.append(ind) 
end_time = time.time()
elapsed_time = end_time - start_time
print('Execution time:', elapsed_time, 'seconds')

Execution time: 252.28145289421082 seconds


In [44]:
dog_dupl.iloc[f_list].shape

(3366, 40)